# Import relevant packages

In [323]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt
import yfinance as yf
import ta
from binance.client import Client
import calendar
client = Client()

# Get Data Function

In [324]:
# Download data
def getdata(symbol, start, end):
    frame = pd.DataFrame(client.get_historical_klines(symbol,'1h', start, end))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame.set_index('Time', inplace = True)
    frame.index = pd.to_datetime(frame.index, unit = 'ms')
    frame = frame.astype(float)
    return frame

In [322]:
data_start_date = '2022-01-01' # Change this variable to get more data
data_end_date = '2023-03-31'

df = getdata('BTCUSDT', data_start_date, data_end_date)
df.info()
df.tail()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10896 entries, 2022-01-01 00:00:00 to 2023-03-31 00:00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    10896 non-null  float64
 1   High    10896 non-null  float64
 2   Low     10896 non-null  float64
 3   Close   10896 non-null  float64
 4   Volume  10896 non-null  float64
dtypes: float64(5)
memory usage: 510.8 KB


,Open,High,Low,Close,Volume
Time,,,,,
2023-03-30 20:00:00,27986.10,28179.70,27900.00,28149.80,3422.58634
2023-03-30 21:00:00,28149.80,28197.96,28012.10,28068.50,1900.85772
2023-03-30 22:00:00,28068.50,28140.00,27786.66,27898.59,2716.85368
2023-03-30 23:00:00,27898.59,28055.00,27841.47,28028.53,1986.44740
2023-03-31 00:00:00,28028.53,28338.34,27916.34,28249.17,4149.41210


# Get Levels Function


In [325]:
def get_levels(date):
    ratios = [-0.618 , 0.618, 1.618]
    series = df.loc[date:][1:2].squeeze()
    diff = series.High - series.Low
    levels = [i * diff + series.Low for i in ratios]
    
    return levels

# Fixing up data & Implementation of backtest

In [326]:
# Get unique dates so that we can iterate through it later
dates = np.unique(df.index.date)

# Create price column such that the price for execution for a datarow is the next data row's open
df['price'] = df.Open.shift(-1)

In [327]:
buys, sells = [], []
trade_dates = []

in_position = False

for date in dates:
    for index, row in df[date:][2:].iterrows():
        if not in_position:
            sl, entry, tp = get_levels(date)
            if row.Close >= entry:
                buys.append(row.price)
                trade_dates.append(date)
                in_position = True
        
        if in_position:
            if row.Close >= tp or row.Close <= sl:
                sells.append(row.price)
                in_position = False
                break

In [328]:
# Create a df for buying and selling prices
trades = pd.DataFrame([buys, sells])
trades.columns = trade_dates # Set the columns to dates that trades were taken
trades.index = ['Buy', 'Sell'] # Set row index to buy and sell
trades = trades.T # Transpose data

trades['PnL'] = trades.Sell - trades.Buy # Create column with PnL for the trades
trades['PnL_rel'] = trades.PnL / trades.Buy # Create % of profit column


# Results

## Algorithm (From start to end)

In [329]:
# Cumulative of % change
cum_series = (trades.PnL_rel + 1).cumprod()
print('PnL for algo would be: {} %'.format(round((cum_series[-1] - 1)*100, 2)))

PnL for algo would be: 98.61 %


## Holding (From start to end)

In [330]:
buy_price = df.iloc[0].Open
sell_price = df.iloc[-1].Close
PnL_rel = (sell_price - buy_price) / buy_price
print('PnL for holding would be: {} %'.format(round(PnL_rel*100, 2)))

PnL for holding would be: -38.88 %


## Algorithm (Returns per month)

### Cumulative for each Month

In [331]:
# Converting trades to date time format
trades.index = pd.to_datetime(trades.index)

# Evaluating the % change for each month
monthly_PnL_series = (trades.PnL_rel + 1).cumprod().resample('M').last()

# Show cumulative percentage gained continuously for each month with algorithm
print('PnL (Continuous) - ALGO (%) \n--------------')
for date, pnl in monthly_PnL_series.items():
    print('{} {} : {} %'.format(date.year, calendar.month_name[date.month], round((pnl-1) * 100, 2)))

PnL - ALGO (%) 
--------------
2022 January : 7.39 %
2022 February : 9.3 %
2022 March : 29.71 %
2022 April : 25.65 %
2022 May : 30.37 %
2022 June : 42.46 %
2022 July : 58.68 %
2022 August : 52.4 %
2022 September : 46.66 %
2022 October : 47.6 %
2022 November : 48.46 %
2022 December : 51.3 %
2023 January : 72.91 %
2023 February : 77.97 %
2023 March : 98.61 %


### Individual Month

In [366]:
# Resampling by month & then applying cumulative product for each month to find percentage change per month with algorithm
per_month_PnL = trades['PnL_rel'].resample('M').apply(lambda x: (1+x).prod(axis = 0) - 1)

#Show individual percentage change for each month with algorithm
print('PnL (Monthly) - ALGO (%) \n------------------------')
for date, pnl in per_month_PnL.items():
    print('{} {} : {} %'.format(date.year, calendar.month_name[date.month], round(pnl * 100, 2)))

PnL (Monthly) - ALGO (%) 
------------------------
2022 January : 7.39 %
2022 February : 1.78 %
2022 March : 18.67 %
2022 April : -3.13 %
2022 May : 3.76 %
2022 June : 9.28 %
2022 July : 11.38 %
2022 August : -3.96 %
2022 September : -3.76 %
2022 October : 0.64 %
2022 November : 0.58 %
2022 December : 1.92 %
2023 January : 14.28 %
2023 February : 2.93 %
2023 March : 11.59 %


## Holding (Returns per month)

In [332]:
df_ = df.copy()

# Resampling data to return first and last data of month
month_start = df_.resample('M').first()
month_end = df_.resample('M').last()

# Finding the change in price if derivative was held from start to end of month
month_diff = month_end.price - month_start.price

# Percentage change 
p_change = (month_diff / month_start.price) * 100

# Show individual percentage change per month for holding strategy
print('PnL - Holding (%) \n-----------------')
for date, pnl in p_change.items():
    print('{} {} : {} %'.format(date.year, calendar.month_name[date.month], round(pnl, 2)))

PnL - Holding (%) 
-----------------
2022 January : -17.55 %
2022 February : 12.56 %
2022 March : 4.48 %
2022 April : -17.37 %
2022 May : -15.6 %
2022 June : -37.56 %
2022 July : 14.79 %
2022 August : -14.24 %
2022 September : -3.2 %
2022 October : 5.61 %
2022 November : -16.08 %
2022 December : -3.61 %
2023 January : 39.9 %
2023 February : 0.24 %
2023 March : 21.33 %
